<a href="https://colab.research.google.com/github/erictong0/FrederickLab2023/blob/main/Linear%20Regression/Initial%20defining%20regions/230615_OLD_Improved_linear_regression_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bio --upgrade

In [ ]:
import sys
import glob
import math
import argparse
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as pltticker
import csv
from matplotlib import colors
import re
import pandas as pd
from numpy import savetxt
from sklearn.linear_model import LinearRegression
from statistics import mean
import statsmodels.api as sm
from Bio.Data.IUPACData import protein_letters_1to3 as one2three
def main(input_file_path, input_exp, aa, rangeChoices):
  aa1 = aa
  data = []
  with open(input_file_path, mode="r") as infile: #open data file and read contents
    reader = csv.reader(infile, delimiter="\n")
    for row in reader:
      if len(row) == 1:
        data.append(re.split('\t', row[0])[2:])
      else:
        data.append(row)
  def get_row_floats(data, row):
    return list(map(lambda x: float(x[row]), data))
  ca = get_row_floats(data[1:], 2) #def x as Ca shift
  cb = get_row_floats(data[1:], 3) #def y as Cb shift
  c = get_row_floats(data[1:], 4) #def z as CO shift
  n = get_row_floats(data[1:], 5) #def w as N shift
  phi = get_row_floats(data[1:], 0)
  psi = get_row_floats(data[1:], 1)

  data = []
  count = 0
  with open(input_exp, mode="r") as infile: #open data file and read contents
    chunk_size = 1000
    while True:
        chunk = infile.readlines(chunk_size)
        if not chunk:
          break
        for line in chunk:
          reader = csv.reader(infile)
          for row in reader:
            count += 1
            if len(row) == 1:
              data.append(re.split(r',', row[0]))
            else:
              data.append(row)
  def get_row_floats(data, row):
    return list(map(lambda x: float(x[row]), data))
  ca_exp = get_row_floats(data,0) #def x as Ca shift
  intensity = get_row_floats(data,2)



  atom_key={
            "ca":0,
            "cb":1,
            "co":2,
            "n":3,
          }

  nonCys_nonGly_aas={"A", "D", "E", "F", "H", "I", "K", "L", "M",
                    "N", "P", "Q", "R", "S", "T", "V", "Y", "W"}

  nonCys_aas={"A", "D", "E", "F", "G", "H", "I", "K", "L", "M",
              "N", "P", "Q", "R", "S", "T", "V", "Y", "W"}

  na=np.nan

  aa_spec_ranges={ #ranges of interest for chemical shifts
                  #aa1 Ca      Cb      C         N
                  # Ca & Cb adjusted to 17ppm to match Fig5
                 'A':[[11,28],[203,234],[170,184],[112,133]],
                  'R':[[47,64],[22,39],[169,183],[112,133]],
                  'N':[[44,61],[30,47],[168,182],[106,127]],
                  'D':[[45,62],[33,50],[169,183],[109,130]],
  #               'C':[[50,67],[22,39],[168,182],[105,126]],
                  'Q':[[47,64],[22,39],[169,183],[108,129]],
                  'E':[[49,66],[22,39],[169,183],[109,130]],
                  'G':[[36,53],[203,234],[167,181],[ 98,119]],
                  'H':[[49,66],[23,40],[168,182],[107,128]],
                  'I':[[54,71],[31,48],[168,182],[109,130]],
                  'L':[[45,62],[33,50],[170,184],[110,131]],
                  'K':[[49,66],[26,43],[169,183],[109,130]],
                  'M':[[47,64],[26,43],[169,183],[108,129]],
                  'F':[[50,67],[31,48],[168,182],[107,128]],
                  'P':[[57,74],[25,42],[170,184],[ na, na]],
                  'S':[[50,67],[105,139],[55,72],[167,181],[104,125]],
                  'T':[[54,71],[104,138],[60,77],[168,182],[102,123]],
                  'W':[[48,65],[21,38],[169,183],[109,130]],
                  'Y':[[50,67],[30,47],[168,182],[108,129]],
                  'V':[[54,71],[25,42],[169,183],[108,129]],
                  }

  ####Changed this part####
  rchoice = rangeChoices

  ### extract plot subregions ###
  # "r" stands for ramachandran
  # regions: beta-strand, polyproline helix ii
  #          beta-turn I, alpha-helix,
  #          left handed helix (all phi > 0)
  # !!! top and left corner are always >= or <=
  #     unless they reach the bottom/right axis
  #     (i.e. betaB, ppiiB, left)

  #filter beta regions
  mask1 = np.logical_and(np.array(phi) >= rchoice['rbetaA'][0], np.array(phi) <= rchoice['rbetaA'][1])
  mask1 = np.logical_and(mask1, np.array(psi) >= rchoice['rbetaA'][2])
  mask1 = np.logical_and(mask1, np.array(psi) <= rchoice['rbetaA'][3])

  mask2 = np.logical_and(np.array(phi) >= rchoice['rbetaB'][0], np.array(phi) <= rchoice['rbetaB'][1])
  mask2 = np.logical_and(mask1, np.array(psi) >= rchoice['rbetaB'][2])
  mask2 = np.logical_and(mask1, np.array(psi) <= rchoice['rbetaB'][3])

  mask = np.logical_or(mask1, mask2)

  data_beta_ca = np.array(ca)[mask]
  data_beta_cb = np.array(cb)[mask]
  data_beta_c = np.array(c)[mask]
  data_beta_n = np.array(n)[mask]

  #filter ppii regions
  mask1 = np.logical_and(np.array(phi) >= rchoice['rppiiA'][0], np.array(phi) <= rchoice['rppiiA'][1])
  mask1 = np.logical_and(mask1, np.array(psi) >= rchoice['rppiiA'][2])
  mask1 = np.logical_and(mask1, np.array(psi) <= rchoice['rppiiA'][3])

  mask2 = np.logical_and(np.array(phi) >= rchoice['rppiiB'][0], np.array(phi) <= rchoice['rppiiB'][1])
  mask2 = np.logical_and(mask1, np.array(psi) >= rchoice['rppiiB'][2])
  mask2 = np.logical_and(mask1, np.array(psi) <= rchoice['rppiiB'][3])

  mask = np.logical_or(mask1, mask2)

  data_ppii_ca = np.array(ca)[mask]
  data_ppii_cb = np.array(cb)[mask]
  data_ppii_c = np.array(c)[mask]
  data_ppii_n = np.array(n)[mask]

  #filter ddgg
  mask = np.logical_and(np.array(phi) >= rchoice['rdelta'][0], np.array(phi) <= rchoice['rdelta'][1])
  mask = np.logical_and(mask1, np.array(psi) >= rchoice['rdelta'][2])
  mask = np.logical_and(mask1, np.array(psi) <= rchoice['rdelta'][3])

  data_ddgg_ca = np.array(ca)[mask]
  data_ddgg_cb = np.array(cb)[mask]
  data_ddgg_c = np.array(c)[mask]
  data_ddgg_n = np.array(n)[mask]

  #filter turn
  mask1 = np.logical_and(np.array(phi) >= rchoice['rturnI'][0], np.array(phi) < rchoice['ralpha'][0])
  mask1 = np.logical_and(mask1, np.array(psi) >= rchoice['rturnI'][2])
  mask1 = np.logical_and(mask1, np.array(psi) <= rchoice['rturnI'][3])

  mask2 = np.logical_and(np.array(phi) >= rchoice['ralpha'][1], np.array(phi) < rchoice['rturnI'][1])
  mask2 = np.logical_and(mask1, np.array(psi) >= rchoice['rturnI'][2])
  mask2 = np.logical_and(mask1, np.array(psi) <= rchoice['rturnI'][3])

  mask3 = np.logical_and(np.array(phi) >= rchoice['ralpha'][0], np.array(phi) < rchoice['ralpha'][1])
  mask3 = np.logical_and(mask1, np.array(psi) >= rchoice['rturnI'][2])
  mask3 = np.logical_and(mask1, np.array(psi) <= rchoice['ralpha'][2])

  mask4 = np.logical_and(np.array(phi) >= rchoice['ralpha'][0], np.array(phi) < rchoice['ralpha'][1])
  mask4 = np.logical_and(mask1, np.array(psi) > rchoice['ralpha'][3])
  mask4 = np.logical_and(mask1, np.array(psi) <= rchoice['rturnI'][3])

  mask = np.logical_or(mask1, np.logical_or(mask2, np.logical_or(mask3, mask4)))
  data_turn_ca = np.array(ca)[mask]
  data_turn_cb = np.array(cb)[mask]
  data_turn_c = np.array(c)[mask]
  data_turn_n = np.array(n)[mask]

  #filter alph
  mask = np.logical_and(np.array(phi) >= rchoice['ralpha'][0], np.array(phi) < rchoice['ralpha'][1])
  mask = np.logical_and(mask1, np.array(psi) > rchoice['ralpha'][2])
  mask = np.logical_and(mask1, np.array(psi) <= rchoice['ralpha'][3])

  data_alph_ca = np.array(ca)[mask]
  data_alph_cb = np.array(cb)[mask]
  data_alph_c = np.array(c)[mask]
  data_alph_n = np.array(n)[mask]

  #filter left
  mask = np.logical_and(np.array(phi) >= rchoice['rlhelx'][0], np.array(phi) <= rchoice['rlhelx'][1])
  mask = np.logical_and(mask1, np.array(psi) >= rchoice['rlhelx'][2])
  mask = np.logical_and(mask1, np.array(psi) <= rchoice['rlhelx'][3])

  data_left_ca = np.array(ca)[mask]
  data_left_cb = np.array(cb)[mask]
  data_left_c = np.array(c)[mask]
  data_left_n = np.array(n)[mask]


  # create dictionaries of data sets

  cs_ca = ({'beta': data_beta_ca,
            'ppii': data_ppii_ca,
            'ddgg': data_ddgg_ca,
            'left': data_left_ca,
            'turn': data_turn_ca,
            'alph': data_alph_ca,
            })
  cs_cb = ({'beta': data_beta_cb,
            'ppii': data_ppii_cb,
            'ddgg': data_ddgg_cb,
            'left': data_left_cb,
            'turn': data_turn_cb,
            'alph': data_alph_cb,
            })
  cs_co = ({'beta': data_beta_c,
            'ppii': data_ppii_c,
            'ddgg': data_ddgg_c,
            'left': data_left_c,
            'turn': data_turn_c,
            'alph': data_alph_c,
            })

  cs_all = {'ca':cs_ca,
            'cb':cs_cb,
            'c' :cs_co,
            }

  bin_width=0.25
  mtick_spacing=5
  fig, axs = plt.subplots(1, 1, sharey=False, tight_layout=True,figsize=(10,10))

  x_lims = np.array(aa_spec_ranges[aa1][0])
  y_lims = np.array(aa_spec_ranges[aa1][1])
  x_edges = np.arange(x_lims[0],x_lims[1]+bin_width,bin_width) #create an array from one limit to the next using bin widths
  y_edges=np.arange(y_lims[0],y_lims[1]+bin_width,bin_width)
  hw=bin_width/2
  center_bins_x=np.arange(x_lims[0]+hw,x_lims[1]+hw,bin_width)
  center_bins_y=np.arange(y_lims[0]+hw,y_lims[1]+hw,bin_width)
  inputs = []
  h_lists = []
  h_list = []

  for key in cs_ca:
    H, bins, patches = axs.hist(cs_ca[key],bins=(x_edges),density=False,histtype='step')
    for j in range(len(H)):
      a = H[j]  # Maximum value
      b = bins[j]  # Peak position
      fwhm = 1.5  # Full width at half maximum

      # Calculate the standard deviation from the FWHM
      c = fwhm / (2 * np.sqrt(2 * np.log(2)))

      x = np.linspace(b - 3 * c, b + 3 * c, 100)


      # Calculate the y values for the curve using the Gaussian function
      y = a * np.exp(-(x - b)**2 / (2 * c**2))

      histogram, bins, patches = axs.hist(x,
                        bins=x_edges,
                        density=False,
                        weights = y,
                        histtype='step')
      try:
            if np.unique(histogram) != 0:
              pass

      except:
        #print(histogram)

        h_list.append(histogram)
    h_lists = sum(h_list)
    inputs.append(h_lists)

  #print(inputs)
  H_exp, bins, patches = axs.hist(ca_exp,bins=(x_edges),density=False,histtype='step')

  target = (H_exp)
  hists1 = (inputs[0])
  hists2 = (inputs[1])
  hists3 = (inputs[2])
  hists4 = (inputs[3])
  hists5 = (inputs[4])
  hists6 = (inputs[5])
  #print(np.shape(inputs),len(inputs[0]),len(target))
  data = {'x1': list(hists1), 'x2': list(hists2), 'x3': list(hists3), 'x4':list(hists4), 'x5':hists5, 'x6':list(hists6), 'y': list(target) }
  df = pd.DataFrame(data)
  x = df[['x1','x2','x3','x4','x5','x6']]
  y = df['y']

  ###prevents plot from displaying
  plt.close()


  model = sm.GLS(y,x)
  results = model.fit()
  return results.rsquared

#new code starts here


"""
topSep = -140.0 #range -180 : 0
topH = 105.0 #range ~100 : 180

middleH = 45.0 #range botH : topH

botSep = -105.0 #range -180 : 0
botH = -150.0 #range -180 : -100"""
def returnDict1(topSep, topH, middleH, botSep, botH):
  return {
      #upper left
      'rbetaA':(-180.0, topSep, topH, 180.0),
      'rppiiA':( topSep,   0.0, topH, 180.0),

      #Middle
      'rdelta':(-180.0,   0.0,  middleH, 105.0),

      #large middle box
      'rturnI':(-180.0,   0.0, botH,  middleH), # turn is wider alpha

      #smaller tiny box
      'ralpha':(-40, -35, -100, -50), # alph is strictly alpha

      #bottom left
      'rbetaB':(-180.0, botSep,-180.0, botH),
      'rppiiB':(botSep,    0.0,-180.0, botH),

      'rlhelx':(   0.0, 180.0,-180.0, 180.0),
  }

def returnDict2(topSep, topH, middleH, botSep, botH, boxL1, boxL2, boxH1, boxH2):
  return {
      #upper left
      'rbetaA':(-180.0, topSep, topH, 180.0),
      'rppiiA':( topSep,   0.0, topH, 180.0),

      #Middle
      'rdelta':(-180.0,   0.0,  middleH, 105.0),

      #large middle box
      'rturnI':(-180.0,   0.0, botH,  middleH), # turn is wider alpha

      #smaller tiny box
      'ralpha':(boxL1, boxL2, boxH1, boxH2), # alph is strictly alpha

      #bottom left
      'rbetaB':(-180.0, botSep,-180.0, botH),
      'rppiiB':(botSep,    0.0,-180.0, botH),

      'rlhelx':(   0.0, 180.0,-180.0, 180.0),
  }



-120, 150, 0, -120, -90, -40, -40, -90, -45
0.556: -125/145/0/-130/-100


In [ ]:
print(returnDict2(-120, 150, 0, -120, -90, -40, -40, -90, -45))

In [ ]:
bestR = 0
bestDict = None
for topSep in range(-115, -99, 5):
  print(f"Outer: {topSep}: \n")
  for topH in range(130, 146, 5):
    for middleH in range(-20, -4, 5):
      for botSep in range(-145, -124, 5):
        for botH in range(-85, -59, 5):
          #r = main("T_10K.out", "nanodisc.csv", "T", returnDict(-180, 105, 45, -105, -150))
          #print(r)
          theDict = returnDict1(topSep, topH, middleH, botSep, botH)
          r = main("T_10K.out", "nanodisc.csv", "T", theDict)
          if r > bestR:
            print(f"{str(round(r, 3))}: {topSep}/{topH}/{middleH}/{botSep}/{botH}")
            bestR = r
            bestDict = theDict

print(bestDict)
print(f"{bestDict['rbetaA'][1]}/{bestDict['rbetaA'][2]}/{bestDict['rturnI'][3]}/{bestDict['rbetaB'][1]}/{bestDict['rbetaB'][3]}")


In [ ]:
bestR = 0
bestDict = None

for boxL1 in range(-50, -29, 5):
  print(f"{boxL1}: ")
  for boxL2 in range(boxL1+5, -24, 5):
    for boxH1 in range(-100, -84, 5):
      for boxH2 in range(-55, -39, 5):
        theDict = returnDict2(-110, 140, -10, -130, -80, boxL1, boxL2, boxH1, boxH2)
        r = main("T_10K.out", "nanodisc.csv", "T", theDict)
        if r > bestR:
            print(f"{str(round(r, 3))}: {boxL1}, {boxL2}, {boxH1}, {boxH2}")
            bestR = r
            bestDict = theDict
print(bestDict)


random code for things:

tempDict = {
    #upper left
    'rbetaA':(-180.0, -140.0, 105.0, 180.0),
    'rppiiA':( -140.0,   0.0, 105.0, 180.0),

    #Middle
    'rdelta':(-180.0,   0.0,  45.0, 105.0),

    #large middle box
    'rturnI':(-180.0,   0.0,-150.0,  45.0), # turn is wider alpha
    #smaller tiny box
    'ralpha':( -90, -30, -60.0, -20.0), # alph is strictly alpha

    #bottom left
    'rbetaB':(-180.0,-105.0,-180.0,-150.0),
    'rppiiB':(-105.0,   0.0,-180.0,-150.0),

    'rlhelx':(   0.0, 180.0,-180.0, 180.0),
    }

In [ ]:
print("Enter the file path, subplot number, and amino acid, separated by commas, or \'Z\' to quit: ")
quit = False
while not quit:
  inputs = input()
  if inputs.upper() == 'Z':
    quit = True
  else:
    inputs = inputs.split(',')
    input_file_path = inputs[0]
    input_exp = inputs[1]
    aa = inputs[2]
    main(input_file_path, input_exp, aa)
    print("Enter the file path for AA, file path for experimental data, subplot number, and amino acid, separated by commas, or \'Z\' to quit: ")